In [96]:
from parflow.tools.io import write_pfb, read_pfb
import numpy as np
from scipy.optimize import fsolve

#read the original input file
#while writing
#old_satur_data = read_pfb("/home/ms6985/ParFlow-project/pfclm_sc/output_water_balance/PFCLM_SC.out.satur.00000.pfb")
#old_press_data = read_pfb("/home/ms6985/ParFlow-project/pfclm_sc/output_water_balance/PFCLM_SC.out.press.00000.pfb")
#read old
old_satur_data = read_pfb("/home/ms6985/ParFlow-project/pfclm_sc/output_water_balance/ic_saturation_WTD-1_alpha2.pfb")
old_press_data = read_pfb("/home/ms6985/ParFlow-project/pfclm_sc/output_water_balance/ic_pressure_WTD-1_alpha2.pfb")



#van-Genuchten model parameters
alpha_vG = 100  #Set alpha for vanGenuchten model
head_table = -1 #location of water table [m] #-1000.0#-1.0 #-10.0  #


s_s = 1.0; s_r = 0.2; n = 3.0 #van-Genuchten model parameters
sw_vG= lambda h,alpha_vG: (s_s - s_r)/((1 + np.abs(alpha_vG*h)**n)**(1-1/n))+s_r
new_press_data = np.empty_like(old_press_data)

#regressing for new pressure
for idx, init_sat in enumerate(old_satur_data):
    print(init_sat[0][0])
    h_from_sat = lambda h: sw_vG(h,alpha_vG) - init_sat[0][0]
    Initial_head = np.abs(fsolve(h_from_sat,1,xtol=1e-15))
    print(idx,Initial_head)
    new_press_data[idx] = -np.abs(Initial_head)

new_press_data[old_press_data>0] = old_press_data[old_press_data>0]  #populating old pressure data with the same pressure

ic_press_filename=f"/home/ms6985/ParFlow-project/pfclm_sc/output_water_balance/ic_pressure_WTD{head_table}_alpha{alpha_vG}.pfb"
ic_satur_filename=f"/home/ms6985/ParFlow-project/pfclm_sc/output_water_balance/ic_saturation_WTD{head_table}_alpha{alpha_vG}.pfb"

write_pfb(ic_press_filename,new_press_data)
write_pfb(ic_satur_filename,old_satur_data)


1.0
0 [0.5625]
1.0
1 [0.5625]
1.0
2 [0.5625]
1.0
3 [0.5625]
1.0
4 [0.5625]
1.0
5 [0.5625]
1.0
6 [0.5625]
1.0
7 [0.5625]
1.0
8 [0.5625]
0.9997270497887782
9 [0.5625]
0.9885023433995179
10 [0.5625]
0.9459681637419519
11 [0.5625]
0.8667012017999374
12 [0.5625]
0.7657573791143819
13 [0.5625]
0.6645736701487781
14 [0.5625]
0.5765144430932899
15 [0.5625]
0.50529784671145
16 [0.0148]
0.44949762614137945
17 [0.0168]
0.4061503529525522
18 [0.0188]
0.38654995427250766
19 [0.0199]


/tmp/ipykernel_995488/1251142367.py:28: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  Initial_head = np.abs(fsolve(h_from_sat,1,xtol=1e-15))


In [97]:
#np.shape(new_press_data)
#old_press_data - new_press_data
new_press_data
#old_press_data

array([[[ 1.71  ]],

       [[ 0.96  ]],

       [[ 0.66  ]],

       [[ 0.56  ]],

       [[ 0.46  ]],

       [[ 0.36  ]],

       [[ 0.26  ]],

       [[ 0.16  ]],

       [[ 0.06  ]],

       [[-0.5625]],

       [[-0.5625]],

       [[-0.5625]],

       [[-0.5625]],

       [[-0.5625]],

       [[-0.5625]],

       [[-0.5625]],

       [[-0.0148]],

       [[-0.0168]],

       [[-0.0188]],

       [[-0.0199]]])

In [98]:
print(old_satur_data)

[[[1.        ]]

 [[1.        ]]

 [[1.        ]]

 [[1.        ]]

 [[1.        ]]

 [[1.        ]]

 [[1.        ]]

 [[1.        ]]

 [[1.        ]]

 [[0.99972705]]

 [[0.98850234]]

 [[0.94596816]]

 [[0.8667012 ]]

 [[0.76575738]]

 [[0.66457367]]

 [[0.57651444]]

 [[0.50529785]]

 [[0.44949763]]

 [[0.40615035]]

 [[0.38654995]]]


In [99]:
print(old_press_data[:,0,0])

[ 1.71   0.96   0.66   0.56   0.46   0.36   0.26   0.16   0.06  -0.04
 -0.14  -0.24  -0.34  -0.44  -0.54  -0.64  -0.74  -0.84  -0.94  -0.995]


In [100]:
sw_calc = sw_vG(new_press_data,alpha_vG); sw_calc[new_press_data>=0]= 1.0 #new_press_data
#print(sw_calc)
print('The difference in saturation is:', np.linalg.norm(sw_calc-old_satur_data))
#import matplotlib.pyplot as plt
#print(sw_calc-old_satur_data)
print(old_satur_data[:,0,0]-sw_calc[:,0,0])

#endings are great but intermediate saturations are not captured properly

The difference in saturation is: 1.7139728931534666
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 7.99474211e-01 7.88249505e-01 7.45715325e-01
 6.66448363e-01 5.65504541e-01 4.64320832e-01 3.76261605e-01
 0.00000000e+00 5.55111512e-17 0.00000000e+00 0.00000000e+00]
